# Class 6: The Bigger Picture

## Guest speakers

### [Elizabeth Marcello, PhD](https://www.linkedin.com/in/elizabeth-marcello-ph-d-686919b0/)

> Elizabeth is a Senior Research Analyst at [Reinvent Albany](https://reinventalbany.org/). She cut her teeth in New York State policy and politics at Reinvent Albany following several years of working in Senegal and then Kenya. After a hiatus to finish her PhD, Elizabeth returned to Reinvent Albany to work on business subsidy and economic development issues. Elizabeth has a master’s degree from Columbia University’s School of International and Public Affairs and a PhD in urban planning from Columbia’s Graduate School of Architecture, Planning and Preservation.

#### Questions?

### [Joel Natividad](https://www.linkedin.com/in/joelnatividad/)

#### Questions?

## Python beyond data analysis

We've been focusing on using Python and pandas for data analysis. What else is Python used for?

### Data engineering

- Automation / recurring processes
- Copying/moving/processing/publishing data, especially Big Data
- Monitoring/alerting

### Web development

- Building web sites that are interactive (more than just content)
- Forms
- Presenting data
- Workflows, such as:
   - Signing up for things
   - Paying for things

### Machine learning

- Statistics, but _fancy_
- Building models
   - [Examples](https://tfhub.dev/)
- Finding patterns
- Recommendations
- Detection

When people say "artificial intelligence," they usually mean "machine learning."

![Diagram showing what type of machine learning may be useful, if at all](https://mitsloan.mit.edu/sites/default/files/2021-04/machine-learning-infographic_2.jpg)

[Source, with more thorough explanation](https://mitsloan.mit.edu/ideas-made-to-matter/machine-learning-explained)

#### The process

_High-level_

1. Create a model
   1. Gather a bunch of data for training
   1. If supervised machine learning, label it (give it the right answers)
   1. Segment into training and test data
   1. Train the model against the training dataset (have it identify patterns)
   1. Test the model against the test dataset
1. Run against new data
1. If reinforcement learning, model refines itself

You have a head start: [The fundamentals](https://python-public-policy.afeld.me/en/{{school_slug}}/syllabus.html#learning-objectives) are applicable anywhere you're using code.

## Post-{{coding_env_name}}

- {{coding_env_name}} instance (and your copy of files) will be deleted
    - Download things you want to keep, particularly edited notebooks
- Class materials will remain available on [the site](https://python-public-policy.afeld.me/en/{{school_slug}}/) and [through GitHub](https://github.com/afeld/python-public-policy/tree/{{school_slug}})
- [Running notebooks elsewhere](https://python-public-policy.afeld.me/en/{{school_slug}}/resources.html#jupyter-outside-this-course)

## [Course evaluations](https://m.albert.nyu.edu/app/student/nyuCrseEval/crseEval/1228/6312/GP/10)

Please complete before signing off! The are:

- Totally anonymous
- Not visible to me until grades are released
- A big help. Some things I took from the past:
    - Making assignments more rigorous
    - Students are hungry for more
    - People like the in-class exercises

Thanks to the {{assistant_name}}!

## Thank you!

Keep in touch:

- [Email](https://python-public-policy.afeld.me/en/{{school_slug}}/syllabus.html#instructor-information)
- [@aidanfeldman](https://twitter.com/aidanfeldman/) on Twitter